# Modeling M100 Lateness

Here we're trying to model the M100's lateness and simulated crowdedness in the St. Nicholas stop going to Inwood 220 St Via Amsterdam Via Bway. 

We are applying Datacamp's Decision-Tree for Classification

## Table of Contents:
1. [Data Cleaning](#data-cleaning)
1. [Plotting a Chart for Sanity](#plotting-a-chart-for-sanity)
1. [Saving our Progress](#saving-our-progress)
1. [Model Training](#model-training)\*
1. [Data Cleaning](#data-cleaning)\*

\* Not finished yet

Imports

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import KFold, train_test_split

warnings.filterwarnings("ignore")
random_state = 20181112
import datetime, math, glob

Adding data from the M100 csv file.

# Choosing the Best Classifier

We want (a) regressor(s) that can predict the **wait time** and **crowding** of a bus at a specific stop with the inputs **hourly weather** and **time of day**. We would most likely have two models that predict each **wait time** and **crowding**.

Here are our top picks for regressors:

1. Gradient Boosting Machines ***(top pick)***:
    - Why: GBMs are typically a composite model that combines the efforts of multiple weak models to create a strong model, and each additional weak model reduces the mean squared error (MSE) of the overall model. Our goal would be to minimize MSE to increase the accuracy of our predictions.

1. Random Forest:
    - Why: does not suffer from the overfitting like with Decision Trees. Instead of randomly choosing to split from just **hourly weather** and **time of day**, we can have two trees that randomly split from each and find the best model. 

1. Decision Trees:  
    - Reduction in Standard Deviation (metric): This is a regression metric that measures how much we’ve reduced our uncertainty by picking a split point. By picking the best split each time the greedy decision tree training algorithm tries to form decisions with as few splits as possible.  
    - Hyperparameters:   
        * Max depth: Limit our tree to a `n` depth to prevent overfitting.
        

Evaluating our model:

Since we're creating regression models, we are interested in the ***mean squared error*** and ***R Squared***. The lower our ***R Squared*** the more accurate our model. We intend to use **K-fold cross validation** as well as a **holdout set** as we improve our model through hyperparameter tuning. 

    * Preventing 

# Data Cleaning

What we need to do:  

1. Clean and break up the time components (Hour, Mins, Secs) of the following:
    * `RecordedAtTime`
2. Merge and store (we'll merge them based on the hour of the day and the day of the month):
    * Bus
        * `Hour`
        * `Min`
        * `Sec`
        * `Day`
    * Weather
        * `Hour`
        * `HourlyVisibility`
        * `HourlyPrecipitation`
        * `HourlyWindSpeed`
3. Features of interest:
    * `Hour`
    * `Min`
    * `Sec`
    * `Hour`
    * `HourlyVisibility`
    * `HourlyPrecipitation`
    * `HourlyWindSpeed`
4. Prediction result:
    * `Hour`
    * `Min`
    * `Sec`
   

# Saving our Progress/Merging Weather Data

In [46]:
%%capture
weather = pd.read_csv('../../data/1401011_weather_data.csv', error_bad_lines=False)

In [162]:
# weather.tail()

In [48]:
%%capture
m100 = pd.read_csv('../../data/M100_Features.csv', error_bad_lines=False)

In [163]:
m100.tail()

      Unnamed: 0      RecordedAtTime VehicleRef        timeDelta  Day  Hour  \
5393        5393 2017-08-29 19:32:56  NYCT_8363  0 days 00:09:59   29    19   
5394        5394 2017-08-29 19:32:59  NYCT_8389  0 days 00:10:28   29    19   
5395        5395 2017-08-29 19:42:01  NYCT_4375  0 days 02:50:12   29    19   
5396        5396 2017-08-29 19:42:01  NYCT_4375  0 days 00:00:00   29    19   
5397        5397 2017-08-29 19:42:32  NYCT_8375  0 days 02:20:49   29    19   

      Min  Sec     stringTime  
5393   32   56  2017-08-29 19  
5394   32   59  2017-08-29 19  
5395   42    1  2017-08-29 19  
5396   42    1  2017-08-29 19  
5397   42   32  2017-08-29 19  

## Data Cleaning
Credit: Angelika

In [166]:
newWeather = weather[['DATE','HOURLYVISIBILITY', 'HOURLYWindSpeed', 'HOURLYPrecip']]

In [167]:
newWeather = newWeather.dropna()
newWeather = newWeather[~newWeather.HOURLYPrecip.str.contains("T")]
newWeather = newWeather[~newWeather.HOURLYPrecip.str.contains("s")]

In [168]:
newWeather = newWeather[~newWeather.HOURLYVISIBILITY.str.contains("V")]
newWeather.HOURLYVISIBILITY=pd.to_numeric(newWeather.HOURLYVISIBILITY)

In [169]:
'''
    @params:
        weather: weather table
        bus: bus table
''' 

def mergeOnDateTime(bus, weather):
    weather['DATE'] = pd.to_datetime(weather['DATE'])
    bus['RecordedAtTime'] = pd.to_datetime(bus['RecordedAtTime'])
    
    weather['stringTime'] = weather['DATE'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    bus['stringTime'] = bus['RecordedAtTime'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    
    newTable = pd.merge(left=bus, right=weather,  how='inner', on=['stringTime'])
    
    newTable.drop(columns='stringTime', inplace=True, axis=1)
    
    return newTable

In [170]:

# newWeather['Hour'] = pd.to_datetime(weather['DATE']).dt.hour
# newWeather['Day'] = pd.to_datetime(weather['DATE']).dt.day


In [171]:
# Time gate to August

newWeather = newWeather[(newWeather['DATE'] > '2017-08-01') & (newWeather['DATE'] < '2017-09-01')].reset_index().dropna()

In [174]:

# Fix some data types
newWeather['HOURLYPrecip'] = pd.to_numeric(newWeather['HOURLYPrecip'], downcast='float', errors='coerce')
newWeather['HOURLYVISIBILITY'] = pd.to_numeric(newWeather['HOURLYVISIBILITY'], downcast='float', errors='coerce')
# Bound hour of day
# newWeather = newWeather[(newWeather['HOUR'] > 4) & (newWeather['HOUR'] < 20)]
newWeather.reset_index()
newWeather.drop(columns=['index'], inplace=True, axis=1)


KeyError: "['index'] not found in axis"

In [175]:
newWeather.dropna()

                 DATE  HOURLYVISIBILITY  HOURLYWindSpeed  HOURLYPrecip
0    2017-08-01 00:51              10.0              3.0           0.0
1    2017-08-01 01:51              10.0              0.0           0.0
2    2017-08-01 02:51              10.0              0.0           0.0
3    2017-08-01 03:51              10.0              3.0           0.0
4    2017-08-01 04:51              10.0              0.0           0.0
5    2017-08-01 05:51              10.0              0.0           0.0
6    2017-08-01 06:51              10.0              0.0           0.0
7    2017-08-01 07:51              10.0              5.0           0.0
8    2017-08-01 08:51              10.0              6.0           0.0
9    2017-08-01 09:51              10.0              0.0           0.0
10   2017-08-01 10:51              10.0              3.0           0.0
11   2017-08-01 11:51              10.0              5.0           0.0
..                ...               ...              ...           ...
747  2

In [176]:
newWeather.dtypes
newWeather.shape

(759, 4)

In [104]:
m100.dtypes
m100.shape

(5398, 9)

In [111]:
df = mergeOnDateTime(m100, newWeather)

In [112]:
df.head()
df.dtypes

Unnamed: 0                   int64
RecordedAtTime      datetime64[ns]
VehicleRef                  object
timeDelta                   object
Day                          int64
Hour                         int64
Min                          int64
Sec                          int64
DATE                datetime64[ns]
HOURLYVISIBILITY           float32
HOURLYWindSpeed            float64
HOURLYPrecip               float32
dtype: object

In [134]:
df.drop(columns=['Unnamed: 0', 'timeDelta'], inplace=True, axis=1)


In [135]:
df.dropna()
df.reset_index()

      index      RecordedAtTime VehicleRef  Day  Hour  Min  Sec  \
0         0 2017-08-01 00:11:39  NYCT_4349    1     0   11   39   
1         1 2017-08-01 00:21:06  NYCT_4368    1     0   21    6   
2         2 2017-08-01 00:41:29  NYCT_8375    1     0   41   29   
3         3 2017-08-01 00:51:52  NYCT_8375    1     0   51   52   
4         4 2017-08-01 01:01:11  NYCT_8375    1     1    1   11   
5         5 2017-08-01 05:21:36  NYCT_8368    1     5   21   36   
6         6 2017-08-01 05:21:36  NYCT_8368    1     5   21   36   
7         7 2017-08-01 05:21:36  NYCT_8368    1     5   21   36   
8         8 2017-08-01 05:31:39  NYCT_8368    1     5   31   39   
9         9 2017-08-01 05:31:39  NYCT_8368    1     5   31   39   
10       10 2017-08-01 05:31:39  NYCT_8368    1     5   31   39   
11       11 2017-08-01 06:01:49  NYCT_8368    1     6    1   49   
...     ...                 ...        ...  ...   ...  ...  ...   
5715   5715 2017-08-29 18:52:07  NYCT_8398   29    18   52    

In [136]:
df.dropna(how='all')
df.count()

RecordedAtTime      5727
VehicleRef          5727
Day                 5727
Hour                5727
Min                 5727
Sec                 5727
DATE                5727
HOURLYVISIBILITY    5727
HOURLYWindSpeed     5727
HOURLYPrecip        5727
dtype: int64

In [137]:
df.tail()

          RecordedAtTime VehicleRef  Day  Hour  Min  Sec                DATE  \
5722 2017-08-29 19:32:56  NYCT_8363   29    19   32   56 2017-08-29 19:51:00   
5723 2017-08-29 19:32:59  NYCT_8389   29    19   32   59 2017-08-29 19:51:00   
5724 2017-08-29 19:42:01  NYCT_4375   29    19   42    1 2017-08-29 19:51:00   
5725 2017-08-29 19:42:01  NYCT_4375   29    19   42    1 2017-08-29 19:51:00   
5726 2017-08-29 19:42:32  NYCT_8375   29    19   42   32 2017-08-29 19:51:00   

      HOURLYVISIBILITY  HOURLYWindSpeed  HOURLYPrecip  
5722               8.0              8.0          0.02  
5723               8.0              8.0          0.02  
5724               8.0              8.0          0.02  
5725               8.0              8.0          0.02  
5726               8.0              8.0          0.02  

# Model Training I
Adapted from: https://plot.ly/scikit-learn/plot-gradient-boosting-regression/

# Saving/Loading the Model

`pip install joblib`

Credit: https://scikit-learn.org/stable/modules/model_persistence.html

In [ ]:
from joblib import dump, load
dump(gbrt, 'model1.joblib') 

# Model Training II

Adapted from: https://shankarmsy.github.io/stories/gbrt-sklearn.html

In [138]:
#Importing required Python packages 
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
from scipy import sparse 
from sklearn.datasets import make_classification, make_blobs, load_boston, fetch_california_housing 
from sklearn.decomposition import PCA 
from sklearn.model_selection import (ShuffleSplit, train_test_split, 
                                     learning_curve, GridSearchCV)
from sklearn import metrics 
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import GradientBoostingRegressor 
from pprint import pprint 
import pandas as pd 
from pandas.tools.plotting import scatter_matrix 
import urllib 
import requests 
import zipfile 
import seaborn

np.random.seed(sum(map(ord, "aesthetics"))) 
seaborn.set_context('notebook') 
# pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier 
plt.rcParams['figure.figsize'] = (15, 5) # Set some Pandas options 
pd.set_option('display.notebook_repr_html', False) 
pd.set_option('display.max_columns', 40) 
pd.set_option('display.max_rows', 25) 
pd.options.display.max_colwidth = 50 


## Features, Targets and Splitting

In [177]:
features = (['Day', 'Hour', 'Sec', 'Min', 'HOURLYVISIBILITY', 'HOURLYWindSpeed', 'HOURLYPrecip'])

target = (['Min', 'Hour', 'Sec'])

model_df = df[(features)].dropna().reset_index()

X_train, X_test, y_train, y_test = train_test_split(
    model_df[features], 
    model_df[target], test_size=0.2,
    random_state=random_state)

# train_df[target] = y_train
# holdout_df[target] = y_holdout

# train_df.reset_index(inplace=True)
# holdout_df.reset_index(inplace=True)

print(X_train.shape[0], X_train[target].mean())
print(X_test.shape[0], X_test[target].mean())

4581 Min     29.447719
Hour    13.416285
Sec     28.938441
dtype: float64
1146 Min     28.204188
Hour    13.239965
Sec     29.576789
dtype: float64


In [155]:
# cal=fetch_california_housing()

In [156]:
# Split data here
# X_train, X_test, y_train, y_test = train_test_split(train_df, y_train) 

## The Gradient Boosting Regression Tree

In [160]:
gbrt=GradientBoostingRegressor(n_estimators=100) 
train_df.shape

gbrt.fit(X_train, y_train) 
y_pred=gbrt.predict(holdout_df) 

ValueError: bad input shape (4581, 3)

## Designing the model

In [ ]:
 def GradientBooster(param_grid, n_jobs): 
        estimator = GradientBoostingRegressor() 
        #Choose cross-validation generator - let's choose ShuffleSplit which randomly shuffles and selects Train and CV sets 
        #for each iteration. There are other methods like the KFold split. 
        cv = ShuffleSplit(X_train.shape[0], test_size=0.2) 
        
        #Apply the cross-validation iterator on the Training set using GridSearchCV. This will run the classifier on the 
        #different train/cv splits using parameters specified and return the model that has the best results 
        #Note that we are tuning based on the F1 score 2PR/P+R where P is Precision and R is Recall. This may not always be 
        #the best score to tune our model on. I will explore this area further in a seperate exercise. For now, we'll use F1. 
        
        classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs) 
        #Also note that we're feeding multiple neighbors to the GridSearch to try out. 
        #We'll now fit the training dataset to this classifier 
        classifier.fit(X_train, y_train) 
        
        #Let's look at the best estimator that was found by GridSearchCV 
        print("Best Estimator learned through GridSearch") 
        print(classifier.best_estimator_) 
        return cv, classifier.best_estimator_ 

In [ ]:
#Below is a plot_learning_curve module that's provided by scikit-learn. It allows us to quickly and easily visualize how #well the model is performing based on number of samples we're training on. It helps to understand situations such as #high variance or bias. 
#We'll call this module in the next segment. 
print(__doc__) 
import numpy as np 
import matplotlib.pyplot as plt 
# from sklearn import cross_validation 
from sklearn.naive_bayes import GaussianNB 
from sklearn.datasets import load_digits 
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)): 
    """ Generate a simple plot of the test and traning learning curve. 
    
    Parameters 
    ---------- 
    estimator : 
    object type that implements the "fit" and "predict" methods An object of that type which is cloned for 
    each validation. title : string Title for the chart. X : array-like, shape (n_samples, n_features) 
    Training vector, where n_samples is the number of samples and n_features is the number of features. 
    y : 
    array-like, shape (n_samples) or (n_samples, n_features), optional Target relative to X for classification 
    or regression; None for unsupervised learning. ylim : tuple, shape (ymin, ymax), optional Defines minimum 
    and maximum yvalues plotted. cv : integer, cross-validation generator, optional If an integer is passed, 
    it is the number of folds (defaults to 3). Specific cross-validation objects can be passed, 
    see sklearn.cross_validation module for the list of possible objects n_jobs : integer, 
    optional Number of jobs to run in parallel (default 1). """ 
    
    plt.figure() 
    plt.title(title) 
    if ylim is not None: 
        plt.ylim(*ylim) 
    plt.xlabel("Training examples") 
    plt.ylabel("Score") 
    train_sizes, train_scores, test_scores = learning_curve( estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes) 
    train_scores_mean = np.mean(train_scores, axis=1) 
    train_scores_std = np.std(train_scores, axis=1) 
    test_scores_mean = np.mean(test_scores, axis=1) 
    test_scores_std = np.std(test_scores, axis=1) 
    plt.grid() 
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r") 
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g") 
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score") 
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score") 
    plt.legend(loc="best") 
    
    return plt 

In [ ]:
#WARNING - THIS MIGHT TAKE A WHILE TO RUN. TRY ADJUSTING parameters such as n_jobs (jobs to run in parallel, before 
#increasing this make sure your system can handle it), n_iter for ShuffleSplit (in the function definition) and reducing 
#number of values being tried for max_depth/n_estimators. 
#SELECT INTERRUPT IN THE MENU AND PRESS INTERRUPT KERNEL IF YOU NEEDD TO STOP EXECUTION 

param_grid={
    'n_estimators':[100], 
    'learning_rate': [0.1],# 0.05, 0.02, 0.01], 
    'max_depth':[6],#4,6], 
    'min_samples_leaf':[3],#,5,9,17], 
    'max_features':[1.0],#,0.3]#,0.1] 
} 

n_jobs=4 
#Let's fit GBRT to the digits training dataset by calling the function we just created. 

cv,best_est=GradientBooster(param_grid, n_jobs) 

In [ ]:
#OK great, so we got back the best estimator parameters as follows:
print("Best Estimator Parameters")
print("---------------------------")
print("n_estimators:", best_est.n_estimators)
print("max_depth:", best_est.max_depth)
print("Learning Rate:", best_est.learning_rate)
print("min_samples_leaf:", best_est.min_samples_leaf)
print("max_features:", best_est.max_features)

print("Train R-squared:", best_est.score(X_train,y_train))

#Each of these parameters is critical to learning. Some of them will help address overfitting issues as well. For more 
#info about overfitting and regularization, check out the SVM notebook in my Github repos where I provide more info on 
#the subject.


In [ ]:
#The module simply runs the estimator multiple times on subsets of the data provided and plots the train and cv scores.
#Note that we're feeding the best parameters we've learned from GridSearchCV to the estimator now.
#We may need to adjust the hyperparameters further if there is overfitting (or underfitting, though unlikely)
title = "Learning Curves (Gradient Boosted Regression Trees)" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf,
                                      max_features=best_est.max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#Looks like we've done a reasonable job getting about ~0.85 R-squared on the cv set and looks from the learning
#curve that we may be able to do a bit better with more estimators. Although we may need to reduce the learning rate even 
#further to address any overfitting.

In [ ]:
#Let's try one more trick. We'll trim the training set to its most important features and re-train to see if 
#that helps.
title = "Learning Curves (Gradient Boosted Regression Trees)" 

#Dropping all parameters except n_estimators and learning_rate since we're going to trim the features anyway.
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, learning_rate=best_est.learning_rate)

#Calling fit on the estimator so we can transform the X matrices.
estimator.fit(X_train, y_train)

#Trimming feature matrices to include only those features that are more important than the mean of all importances.
# X_train_trim=estimator.transform(X_train, threshold='mean')

# #Trimming test as well in case we end up going with this model as final.
# X_test_trim=estimator.transform(X_test, threshold='mean')

#Re-plotting Learning cruves.
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#So what do we infer from this plot? We seem to have addressed overfitting much better but the overall score of both train
#and cv has gone down considerably, indicating that the features we dropped were actually collectively contributing
#to the model. Let's go back to the first model that the Grid Search returned and run our test scores.


In [ ]:
#Switching back to the best model from gridsearch
estimator = best_est

#Re-fitting to the train set
estimator.fit(X_train, y_train)

#Calculating train/test scores - R-squared value
print("Train R-squared: ", estimator.score(X_train, y_train))
print("Test R-squared: ", estimator.score(X_test, y_test))

#There you have it, our final R-squared on the California housing dataset, 0.82

In [ ]:
#OK let's run through a more complex example this time. We'll explore anonymous loan data provided by lendingclub. 
#We'll try to predict the interest rate for loan applications based on data provided. Let's first download data to 
#a pandas df.

#The Dataset is a zip file. So let's first read in the dataset through requests then pass it on to Pandas through the
#read_csv command
url=requests.get('https://resources.lendingclub.com/LoanStats3c.csv.zip')
z=zipfile.ZipFile(io.StringIO(url.content))

loan=pd.read_csv(z.open('LoanStats3c.csv'), skiprows=1, parse_dates=True, index_col='id')
loanbk=loan.copy() #Backup of the dataframe so we don't have to download data everytime

In [ ]:
#Let's take a quick peek at the dataset
loan.describe()

In [ ]:
#For simplicity, let's first drop nulls in the dataset. axis=1 indicates we'll drop rows not cols.
loan = loan.dropna(axis=0)

In [ ]:
#OK let's take a look at the columns and see if there are any we can drop any before we get started.
loan.columns.values

#There're plenty that don't seem very relevant. Let's drop them.
loan=loan.drop(['member_id', 'grade', 'sub_grade', 'emp_title', 'issue_d',
          'pymnt_plan', 'url', 'desc', 'title', 'initial_list_status',
          'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
          'policy_code', 'emp_length', 'addr_state','zip_code'], axis=1)

#Check the data dictionary for this dataset at https://resources.lendingclub.com/LCDataDictionary.xlsx for more details

In [ ]:
# Get rid of non-numeric values throughout the DataFrame:
for col in loan.columns.values:
  loan[col] = loan[col].replace('[^0-9]+.-', '', regex=True)
loan.head(2)

In [ ]:
#Remove % symbol from the interest rate & revolving utilization
loan.int_rate=loan.int_rate.str.split('%',1).str[0]
loan.revol_util=loan.revol_util.str.split('%',1).str[0]

#Remove "months" from the loan period
loan.term=loan.term.str.split(' ',2).str[1]

loan.head(2)

In [ ]:
#Let's change the Income Verified column, which currently has textual labels to numeric.
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
loan.is_inc_v = le.fit_transform(loan.is_inc_v.values)
loan.home_ownership=le.fit_transform(loan.home_ownership.values)
loan.loan_status=le.fit_transform(loan.loan_status.values)
loan.purpose=le.fit_transform(loan.purpose.values)

#Finally let's be sure we convert all fields to numeric
loan=loan.convert_objects(convert_numeric=True)

loan.head(2)

In [ ]:
#OK great, let's now get our X and y. We know that interest rate is y.
#Pandas is fantastic, all you need to do is use .values to get the data in numpy format
y=loan.int_rate.values

#Let's remove y from the df so we can get X
del loan['int_rate']
X=loan.values

#Now, the train test split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
#Great, in no time we have grabbed an unknown dataset from the web, munged it using Pandas and now have ready-to-go
#training and test numpy arrays for running the GBRT regressor. Let's go!

#WARNING - THIS MIGHT TAKE A WHILE TO RUN. TRY ADJUSTING parameters such as n_jobs (jobs to run in parallel, before 
#increasing this make sure your system can handle it), n_iter for ShuffleSplit (in the function definition) and reducing 
#number of values being tried for max_depth/n_estimators.

#SELECT INTERRUPT IN THE MENU AND PRESS INTERRUPT KERNEL IF YOU NEEDD TO STOP EXECUTION

param_grid={'n_estimators':[100],#,500,1000],
            'learning_rate': [0.1,0.05,0.02],# 0.01],
            'max_depth':[4,6], 
            'min_samples_leaf':[3,5,9,17], 
            'max_features':[1.0,0.3,0.1]
           }
n_jobs=4

#Let's fit GBRT to the digits training dataset by calling the function we just created.
cv,best_est=GradientBooster(param_grid, n_jobs)

In [ ]:
#OK great, so we got back the best estimator parameters as follows:
print("Best Estimator Parameters")
print("---------------------------")
print ("n_estimators:", best_est.n_estimators)
print ("max_depth:", best_est.max_depth)
print ("Learning Rate:", best_est.learning_rate)
print ("min_samples_leaf:", best_est.min_samples_leaf)
print ("max_features:", best_est.max_features)

print ("Train R-squared:", best_est.score(X_train,y_train))

#The training R-Squared is almost 1.0 which indicates we can understand 99% of the variance in the data as well as
#there's a chance we might overfit. Let's see with the learning curves below.


In [ ]:
#OK we'll now call the plot_learning_curve module by feeding it the estimator (best estimator returned from GS) 
#and train/cv sets.

#The module simply runs the estimator multiple times on subsets of the data provided and plots the train and cv scores.
#Note that we're feeding the best parameters we've learned from GridSearchCV to the estimator now.
#We may need to adjust the hyperparameters further if there is overfitting (or underfitting, though unlikely)
title = "Learning Curves (Gradient Boosted Regression Trees)" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf,
                                      max_features=best_est.max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#OK yes, there is some overfitting there. We can see the training scores in red almost close to 1.0 and the cv scores
#trying its best to reach it as the number of examples increases. This is what happens during overfitting. To address
#overfitting, GBRT basically has the following parameters we can fine tune: Learning Rate, Max Depth, Min Samples leaf and
#Max features.

In [ ]:
#the typical recommended values of Max depth is 4 to 6, so lets leave it at 4. Let's try increasing the min
#samples leaf parameter, this basically enforces a lower bound on the number of samples in any given leaf.
min_samples_leaf=9

title = "Learning Curves (Gradient Boosted Regression Trees), min_samples_leaf=9" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=best_est.learning_rate, min_samples_leaf=min_samples_leaf,
                                      max_features=best_est.max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

In [ ]:
#Let's try reducing the max features parameter. This enforces an upper bound of the maximum number of features to use
#for training. It's supposed to work well when n_features>30. We'll also remove min samples leaf for this run.
max_features=0.5

title = "Learning Curves (Gradient Boosted Regression Trees), max_features=50%" 
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=best_est.learning_rate, max_features=max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#Nope that didn't quite improve the cv score either. What happens if we reduce learning rate?

In [ ]:
#The lower the learning rate is the more the number of trees we need to train. This is because the rate at which we train
#is simply, well, reduced.
learning_rate=.01
n_estimators=1000

title = "Learning Curves (Gradient Boosted Regression Trees), 1000 Trees at learning rate .01"
estimator = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=learning_rate, min_samples_leaf=best_est.min_samples_leaf,
                                      max_features=best_est.max_features)
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#Perhaps that improved it a tiny little bit.

In [ ]:
#Before we try anything else, I would like to explore one of the beautiful advantages of growing trees. And that is to
#capture feature importances. Now that we have a publicly available loan application collection (though anonymous), it makes
#me really curious to see what impacts the interest rate for a loan application the most.

#Let's take a look

#Calling fit on the estimator so we can look at feature_importances.
estimator.fit(X_train, y_train)

# Calculate the feature ranking - Top 10
importances = estimator.feature_importances_
indices = np.argsort(importances)[::-1]

print "Lending Club Loan Data - Top 10 Important Features\n"

for f in range(10):
    print("%d. %s   (%f)" % (f + 1, loan.columns[indices[f]], importances[indices[f]]))
    
#Plot the feature importances of the forest
indices=indices[:10]
plt.figure()
plt.title("Top 10 Feature importances")
plt.bar(range(10), importances[indices],
       color="r", align="center")
plt.xticks(range(10), loan.columns[indices], fontsize=14, rotation=45)
plt.xlim([-1, 10])
plt.show()

#Mean Feature Importance
print "Mean Feature Importance %.6f" %np.mean(importances)

#Interesting, the total amount of interest received to date is the top most influencer for getting a better interest rate.
#Good for the lenders eh? Pay more interest, we'll give you a cut on the interest rate. Of course!

In [ ]:
#Can we actually trim (like before) and get a better result? Perhaps not, but who's to stop us from trying.
title = "Learning Curves (Gradient Boosted Regression Trees) - Trimmed features to > 1% importance" 

#Dropping all parameters except n_estimators and learning_rate since we're going to trim the features anyway.
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, learning_rate=best_est.learning_rate)

#Calling fit on the estimator so we can transform the X matrices.
estimator.fit(X_train, y_train)

#Trimming feature matrices to include only those features that are more important than the mean of all importances.
X_train_trim=estimator.transform(X_train, threshold=.01)

#Trimming test as well in case we end up going with this model as final.
X_test_trim=estimator.transform(X_test, threshold=.01)

#Re-plotting Learning cruves.
plot_learning_curve(estimator, title, X_train, y_train, cv=cv, n_jobs=n_jobs)
plt.show()

#Nope, the curve looks like it overfits less, but look at the cv score, in all our fancy attempts it never really crossed
#that ~0.8 R-squared barrier. That tells me, we actually have a decent model at hand and also that a 0.9+ R-squared value is
#not always possible, atleast in real time. Let's wrap this up.

In [ ]:
estimator = GradientBoostingRegressor(n_estimators=best_est.n_estimators, max_depth=best_est.max_depth,
                                      learning_rate=best_est.learning_rate, min_samples_leaf=best_est.min_samples_leaf,
                                      max_features=best_est.max_features)
estimator.fit(X_train, y_train)

print("Final Estimator Parameters")
print("---------------------------")
print("n_estimators:", best_est.n_estimators)
print("max_depth:", best_est.max_depth)
print("Learning Rate:", best_est.learning_rate)
print("min_samples_leaf:", best_est.min_samples_leaf)
print("max_features:", best_est.max_features)
print("Final Train R-squared:", estimator.score(X_train, y_train))
print("Final Test R-squared:", estimator.score(X_test, y_test))